In [1]:
# pip install sentence_transformers==2.7.0

In [2]:
from sentence_transformers import SentenceTransformer

/home/codespace/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

## Q1. Getting the embeddings model
### What's the first value of the resulting vector?

In [4]:
user_question = "I just discovered the course. Can I still join it?"

In [6]:
v= embedding_model.encode(user_question)

In [7]:
len(v)

768

In [8]:
v[0]

0.078222655

## Prepare the documents

In [10]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [17]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [16]:
len(documents)

948

In [14]:
# Desired course to filter
desired_course = 'machine-learning-zoomcamp'

# Filter documents
filtered_documents = [doc for doc in documents if doc['course'] == desired_course]

In [15]:
len(filtered_documents)

375

## Q2. Creating the embeddings
###  What's the shape of X? (X.shape). Include the parantheses.

In [18]:
import numpy as np
from tqdm.auto import tqdm

In [20]:
embeddings = []

for doc in tqdm(filtered_documents):
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    embeddings.append(embedding_model.encode(qa_text))

X = np.array(embeddings)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [01:04<00:00,  5.84it/s]


In [21]:
X.shape

(375, 768)

## Q3. Search
### What's the highest score in the results?

In [22]:
scores = X.dot(v)

In [25]:
max_score = np.max(scores)

In [26]:
max_score

0.6506573